In [41]:
import os
from gurobipy import Model, GRB
import matplotlib.pyplot as plt

def read_exams(file_path):
    exams = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) != 2:
                print(f"Skipping malformed line in exams file: {line.strip()}")
                continue
            exam_id, num_students = map(int, parts)
            exams[exam_id] = num_students
    return exams

def read_time_slots(file_path):
    with open(file_path, 'r') as file:
        return int(file.readline().strip())

def read_students(file_path):
    students = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.split()
            if len(parts) != 2:
                print(f"Skipping malformed line in students file: {line.strip()}")
                continue
            student_id = parts[0]
            exam_id = int(parts[1])
            if student_id not in students:
                students[student_id] = []
            students[student_id].append(exam_id)
    return students

def build_conflicts(students):
    conflicts = {}
    for exams in students.values():
        for i in range(len(exams)):
            for j in range(i + 1, len(exams)):
                e1, e2 = exams[i], exams[j]
                if (e1, e2) not in conflicts and (e2, e1) not in conflicts:
                    conflicts[(e1, e2)] = 0
                conflicts[(e1, e2)] += 1
    return conflicts


In [42]:
def solve_etp(exams, students, time_slots, conflicts, time_limit=600):
    model = Model('ETP')
    
    # Variables
    x = model.addVars(exams.keys(), range(1, time_slots + 1), vtype=GRB.BINARY, name='x')
    max_distance = model.addVar(vtype=GRB.INTEGER, name='max_distance')
    back_to_back = model.addVar(vtype=GRB.INTEGER, name='back_to_back')
    
    # Constraints
    # Each exam must be scheduled exactly once
    for e in exams.keys():
        model.addConstr(sum(x[e, t] for t in range(1, time_slots + 1)) == 1)
    
    # At most 3 conflicting pairs in the same time slot
    for t in range(1, time_slots + 1):
        model.addConstr(sum(x[e1, t] * x[e2, t] for (e1, e2), _ in conflicts.items()) <= 3)
    
    # At most 3 consecutive time slots can have conflicting exams
    conflict_in_slot = model.addVars(range(1, time_slots + 1), vtype=GRB.BINARY, name='conflict_in_slot')
    for t in range(1, time_slots + 1):
        model.addConstr(conflict_in_slot[t] >= sum(x[e1, t] * x[e2, t] for (e1, e2), _ in conflicts.items()) / len(exams))
    
    for t in range(1, time_slots - 2):
        model.addConstr(sum(conflict_in_slot[t + i] for i in range(3)) <= 3)
    
    # If two consecutive time slots contain conflicting exams, then no conflicting exam can be scheduled in the next 3 time slots
    for t in range(1, time_slots - 4):
        model.addConstr((conflict_in_slot[t] + conflict_in_slot[t+1]) <= 2 * (1 - conflict_in_slot[t+2]))
    
    # Add bonus for no conflicting exams in 6 consecutive time slots
    bonus_vars = model.addVars(range(1, time_slots - 5 + 1), vtype=GRB.BINARY, name='bonus')
    for t in range(1, time_slots - 5 + 1):
        model.addConstr(bonus_vars[t] <= 1 - sum(x[e1, t+i] * x[e2, t+i] for (e1, e2), _ in conflicts.items() for i in range(6)))
    
    bonus = sum(bonus_vars)
    
    # Maximize the distance between conflicting exams
    for (e1, e2), _ in conflicts.items():
        for t1 in range(1, time_slots + 1):
            for t2 in range(1, time_slots + 1):
                if t1 != t2:
                    model.addConstr(max_distance >= (t2 - t1) * (x[e1, t1] * x[e2, t2] + x[e2, t1] * x[e1, t2]))
    
    # Minimize the number of students with back-to-back exams
    model.addConstr(back_to_back == sum(x[e1, t] * x[e2, t+1] for (e1, e2), _ in conflicts.items() for t in range(1, time_slots)))
    
    # Objective: Minimize the penalty for close conflicts and include equity measures
    penalty = sum(2**(5-i) * num_students * (x[e1, t] * x[e2, t+i])
                  for (e1, e2), num_students in conflicts.items()
                  for t in range(1, time_slots - 5 + 1)
                  for i in range(1, 6))
    
    # Add equity measures to the objective function
    model.setObjective(penalty + 1000 * back_to_back - max_distance + bonus, GRB.MINIMIZE)
    
    # Set a time limit
    model.setParam('TimeLimit', time_limit)
    
    # Optimize the model
    model.optimize()
    
    # Check and return results
    if model.Status == GRB.OPTIMAL or model.Status == GRB.TIME_LIMIT:
        solution = {e: t for e in exams.keys() for t in range(1, time_slots + 1) if x[e, t].X > 0.5}
        return solution, model.ObjVal, max_distance.X, back_to_back.X
    else:
        return None, None, None, None


In [43]:
def validate_solution(solution, exams, students, conflicts, time_slots):
    # Validate each exam is scheduled exactly once
    scheduled_exams = set(solution.keys())
    all_exams = set(exams.keys())
    if scheduled_exams != all_exams:
        print("Validation Failed: Some exams are not scheduled exactly once.")
        return False

    print("All constraints are satisfied.")
    return True

def calculate_objective_value(solution, conflicts):
    penalty = 0
    for (e1, e2), num_students in conflicts.items():
        distance = abs(solution[e1] - solution[e2])
        if 1 <= distance <= 5:
            penalty += 2**(5 - distance) * num_students
    return penalty

def visualize_timetable(solution, time_slots):
    exams_per_slot = {}
    for exam, slot in solution.items():
        if slot not in exams_per_slot:
            exams_per_slot[slot] = []
        exams_per_slot[slot].append(exam)

    fig, ax = plt.subplots()
    ax.set_title('Examination Timetable')
    ax.set_xlabel('Time Slots')
    ax.set_ylabel('Exams')
    
    for slot in range(1, time_slots + 1):
        exams = exams_per_slot.get(slot, [])
        for exam in exams:
            ax.plot(slot, exam, 'ro')
    
    plt.show()

def write_output(filename, solution):
    with open(filename, 'w') as file:
        if not solution:
            file.write("UNFEASIBLE\n")
        else:
            for exam, time_slot in sorted(solution.items()):
                file.write(f"{exam} {time_slot}\n")


In [44]:
def main(instance, time_limit=600):
    # Define the instance to solve
    exam_file = f"input/{instance}.exm"
    time_slots_file = f"input/{instance}.slo"
    student_file = f"input/{instance}.stu"
    output_file = f"output/{instance}.out"
    
    # Read input files
    try:
        exams = read_exams(exam_file)
        time_slots = read_time_slots(time_slots_file)
        students = read_students(student_file)
    except Exception as e:
        print(f"Error reading input files: {e}")
        return
    
    # Build conflicts
    conflicts = build_conflicts(students)
    
    # Solve ETP with equity measures and additional restrictions
    solution, obj_val, max_distance, back_to_back = solve_etp(exams, students, time_slots, conflicts, time_limit=time_limit)
    
    # Validate solution
    if solution:
        print(f"Optimal solution found with objective value: {obj_val}")
        print(f"Maximum distance between conflicting exams: {max_distance}")
        print(f"Number of students with back-to-back exams: {back_to_back}")
        for exam, time_slot in solution.items():
            print(f"Exam {exam} is scheduled at time slot {time_slot}")

        # Validate constraints
        is_valid = validate_solution(solution, exams, students, conflicts, time_slots)
        
        if is_valid:
            # Calculate and print the objective value
            calculated_obj_val = calculate_objective_value(solution, conflicts)
            print(f"Calculated objective value: {calculated_obj_val}")

            # Visualize the timetable
            visualize_timetable(solution, time_slots)
            
            # Write the output to file
            write_output(output_file, solution)
    else:
        print("No feasible solution found or optimization time limit reached")
        write_output(output_file, None)

# Run the main function with the desired instance
instance_name = "instance01"
main(instance_name, time_limit=600)  # Set the time limit to 600 seconds (10 minutes)


Skipping malformed line in exams file: 
Set parameter TimeLimit to value 600
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 215 rows, 3840 columns and 3903 nonzeros
Model fingerprint: 0xdb6f562f
Model has 376480 quadratic objective terms
Model has 1976579 quadratic constraints
Variable types: 0 continuous, 3840 integer (3838 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  QMatrix range    [6e-03, 2e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+03]
  QObjective range [2e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
  QRHS range       [1e+00, 3e+00]
Presolve removed 18 rows and 17 columns (presolve time = 16s) ...
Presolve removed 18 rows and 17 columns
Presolve time: 15.57s
Presolved: 3252102 ro